# Working with Raster data in Python

## Learning Objectives

 * Read spatial raster formats from web services and files
 * Write spatial raster formats to disk
 * Apply basic operations on raster data
 * Plot spatial raster data with matplotlib and rasterio
 * Work with categorical rasters


## Raster data

Raster data is stored as a grid of values which are rendered as pixels on a map. Each pixel value represents an area on the Earth’s surface.
Digital photographs are also a raster. A geospatial raster includes spatial information that connects the data to a particular location on Earth. 

![](./img/raster-concept.png)
Source: Colin Williams, NEON

## All good with Seurasaaris trees?

As researchers we often wonder about the weirdest things. 
In this lesson we will wonder about how the trees of [Seurasaari](https://www.hel.fi/helsinki/en/culture/recreation/in-helsinki/seurasaari) (small island in western Helsinki) were doing in end of summer 2021.

And since we do not just wonder but also want to really know, we will see if we can answer this question using some freely available data and open source tools.

Our data: \
-> Earth Observation (i.e. satellite remote sensing) data: we want to know the situation at a specific time and place. \
-> multispectral remote sensing data: we are interested in spectral information rather than structural. \
-> [Sentinel-2 data](https://sentinel.esa.int/web/sentinel/missions/sentinel-2): Seurasaari is rather small and Sentinel-2 provides up to 10 meter spatial resolution (pixel size). It also provides data in 12 bands (wavelengths intervals) and the bottom-of-atmosphere (L2A) product is corrected for atmospheric disturbances, which is directly provided by the European Space Agency (ESA). The Landsat mission provides similar datasets with a slightly lower spatial and temporal resolution, but a longer time series. 

Our (main) tool for today:
[Rasterio](https://rasterio.readthedocs.io/en/latest/intro.html) Python package (depends on [GDAL](https://gdal.org/)). 

Let's import rasterio library (https://rasterio.readthedocs.io/en/latest/) and a small custom toolbox (see seurasaari_toolbox.py) that will help us with the tasks.

In [ ]:
#In March 2024 course this next line is needed to install two missing packages
#!pip install rasterstats xlrd

In [ ]:
import rasterio
print(rasterio.__version__)

import seurasaari_toolbox as stb

### Getting the data

Sentinel-2 data can be downloaded from numerous sources. Depending on which data product is needed and of what area (and time), different sources may be suitable. The most direct source is the [Copernicus Data Space Ecosystem](https://dataspace.copernicus.eu/). It provides multiple ways to access and download the data. It also includes instructions for many tools to browse and download Sentinel-2 data. Check out our [Earth Observation guide](https://docs.csc.fi/support/tutorials/gis/eo_guide/) to find out more about the CDSE and other tools and platforms

Lucky for us, we have a pre-processed geotiff file on Allas ( url: https://a3s.fi/gis-courses/pythongis_2022/S2B_RGBNIR_20210926_Helsinki.tif ) that includes 4 band (red, green, blue, near infrared) Sentinel-2 data of 26.09.2021. You can see the preprocessing steps done to this file in `prepare_data.py`.
Let's get it and have a look. First we utilize the toolbox to download the data from Allas and open it with rasterio.

In [ ]:
s2url = "https://a3s.fi/gis-courses/pythongis_2022/S2B_RGBNIR_20210926_Helsinki.tif"
stb.download_data(s2url)

s2file = "./data/S2B_RGBNIR_20210926_Helsinki.tif"
s2open = rasterio.open(s2file)


> Note: For data on Allas you could also access the data directly without specifically downloading it using vsicurl (virtual system interface for Curl)
like so: `rasterio.open("/vsicurl/https://a3s.fi/gis-courses/pythongis_2022/S2B_RGBNIR_20210926_Helsinki.tif")`

### What metadata is available?

In [ ]:
s2open.meta

Affine transformation tells about how the pixels are mapped to a geospatial location with origin, pixel size and rotation of the raster in the geographic coordinate system. 
> Note that bounds, crs and transformation are interrelated and updating one will also affect the others.

The four bands available to us (1:blue, 2:green, 3:red, 4: near infrared) can tell us different things.

![](./img/vegetation_reflectance.png)

Credit: Physicsopenlab, http://physicsopenlab.org/wp-content/uploads/2017/01/veg.gif


### Visualizing the near infrared band

It looks like the near infrared can tell us something about vegetation. Let's visualize it! Rasterio provides its own plotting function. With show we can visualize different bands.


In [ ]:
from rasterio.plot import show

show((s2open,4))

Great, we can see that our data shows Helsinki, and that we have a range of different reflectances in the near infrared band.

### Common false color image

But there is better ways to look at vegetation. Let's for example have a look at a "common false color image" which utilizes the nir, green and red band to be represented as red, green and blue bands for the image.
Let's read the raster values into numpy arrays and check what we got.

In [ ]:
nir = s2open.read(4)
red = s2open.read(3)
green = s2open.read(2)

print(type(nir))
print(nir.dtype)

"Uint" or unsigned integers are chosen for Sentinel-2 data to conserve storage space. To get the reflectance, every pixel value is to be divided by 10000. Sentinel-2 L2A products no-data value is 0. We can set these 0 values to 'nan' (not a number) instead. These are common processes for earth observation data. However, the correction factor and nodata values might be different for different sensors. You can usually find this information from the data products metadata.
To rescale the pixelvalues to values between 0 and 1 and replace the no data value 0 with 'nan', we need numpy.

### Rescaling and replacing no data values

In [ ]:
import numpy as np

# rescaling 
red = red / 10000
green = green / 10000
nir = nir / 10000

# replace 0 with nan
red[red == 0] = np.nan
green[green == 0] = np.nan
nir[nir == 0] = np.nan
nir.dtype

Now let's take a look at the false color image. We will use pyplot imshow for plotting. In case of multiband images, it interprets the first band as red, second as green and third as blue.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

For the falsecolor image, we want nir -> red, red -> green and green -> blue and stack the bands into one multidimensional array.

In [ ]:
falsecolor_stack = np.dstack((nir, red, green))

falsecolor_stack.shape

Then we can plot the array with imshow:

In [ ]:
plt.imshow(falsecolor_stack)

We divided the values by 10000 to get values between 0 and 1. However, imshow warns us that it had to clip some values for the plot. Let's see what went wrong by calculating the minimum, maximum and mean of the red band.

In [ ]:
print(np.nanmin(red), "-", np.nanmax(red), "mean:", np.nanmean(red))

### Fixing the value range

It seems like we do have some values outside of the [0,1] range. In our case, these values are likely some artifacts from preprocessing or atmospheric correction. To avoid this warning , we can set all values above 1 to 1. Note that this is based on an assumption and these new 1 values are not real. It would be more correct to set these values to nan. We can again use numpy to adjust the values. Then we need to stack the new arrays again and check their minimum, maximum and mean values.

In [ ]:
red[red>1] =1
green[green>1] = 1
nir[nir>1] =1

falsecolor_stack = np.dstack((nir, red, green))

print(np.nanmin(red), "-", np.nanmax(red), "mean:", np.nanmean(red))

That looks better.

### Histogram 

Let's also look at how our values are distributed with [rasterios histogram plotting function](https://rasterio.readthedocs.io/en/latest/api/rasterio.plot.html#rasterio.plot.show_hist) :

In [ ]:
from rasterio.plot import show_hist

In [ ]:
show_hist(falsecolor_stack, bins=50, stacked=False, alpha=0.9, histtype="stepfilled")

**Check your understanding:**

* Something seems off about the histogram, can you find out what and fix it?
([Hint](https://rasterio.readthedocs.io/en/latest/topics/plotting.html): check the shape of stack and how rasterio stores it )

* The basic output of the histogram function is not very nice and even wrong in one place, can you do better?
(Hint: show_hist also takes for example matplotlib axes (`ax=`) as optional parameters)

In [ ]:
# %load cyu1_solution.txt

### Stretching the value range

It looks like we have a lot of values below 0.5, but using the whole 0-1 range to plot the false color image. This may be the reason it appears so dark.
Let's utilize some image processing (from the toolbox) to disregard very large and very small values and stretch the remaining values across the histogram for visualization purposes.

In [ ]:
stretchnir = stb.stretch(nir)
stretchred = stb.stretch(red)
stretchgreen = stb.stretch(green)

print("before")
print(np.nanmin(red), "-", np.nanmax(red), "mean:", np.nanmean(red))
print("after")
print(np.nanmin(stretchred), "-", np.nanmax(stretchred), "mean:", np.nanmean(stretchred))

That looks better, let's plot it! 
First we need to stack the stretched bands again, then we can again use imshow to plot the stretched stack. 

In [ ]:
stretchstack = np.dstack((stretchnir, stretchred, stretchgreen))

plt.imshow(stretchstack)

Much better. Now we can see how green Helsinki is.

## Cropping the image

However, we cannot see really well what is going on on Seurasaari.

Since we are not interested in the rest of Helsinki, let's crop Seurasaari. 
First, let's get a polygon framing Seurasaari using osmnx tool and store it as geodataframe. You can find the keywords that can be used for string search from OSM: https://nominatim.openstreetmap.org/ui/search.html

In [ ]:
import osmnx as ox

seurasaari_string = "Seurasaari, Helsinki, Finland"
seurasaari_df = ox.geocode_to_gdf(seurasaari_string)
seurasaari_df

#### Reprojecting

For the cropping operation we need both the Sentinel-2 data and the Seurasaari polygon in the same Coordinate Reference System (CRS). First we check the CRS of the Seurasaari polygon and our Sentinel-2 scene.

In [ ]:
print(seurasaari_df.crs)
print(s2open.crs)

Unfortunately they do not share the same CRS, so let's reproject the Seurasaari polygon:

In [ ]:
seurasaari_df = seurasaari_df.to_crs(crs=s2open.crs)

### Reformatting

Now we need to get the format of the seurasaari polygon into something usable for rasterio (https://rasterio.readthedocs.io/en/latest/api/rasterio.mask.html#rasterio.mask.mask).
Let's use the toolbox to get the coordinates in the correct format, a geoJSON-like dictionary:

In [ ]:
seurasaari_coords = stb.getFeatures(seurasaari_df)

### Masking

Now we want to use the mask functionality of rasterio to crop the Sentinel-2 scene to Seurasaari. 

In [ ]:
import rasterio.mask

seurasaariS2_array, seurasaariS2_transform = rasterio.mask.mask(dataset=s2open, shapes=seurasaari_coords, crop=True)

#seurasaariS2_array.shape

### Storing

Let's store this cropped version as geotiff. 
First we need to copy the metadata as key-value pairs from the original geotiff and adjust the metadata that we changed.

In [ ]:
seurasaariS2_meta = s2open.meta.copy()

seurasaariS2_meta.update({"height": seurasaariS2_array.shape[1],
                 "width": seurasaariS2_array.shape[2],
                 "transform": seurasaariS2_transform }
                )

seurasaariS2_file = "./data/S2B_RGBNIR_20210926_Seurasaari.tif"

with rasterio.open(seurasaariS2_file, "w", **seurasaariS2_meta) as dest:
        dest.write(seurasaariS2_array)

Now we are done with the Sentinel-2 data of Helsinki. Let's not forget to close it.

In [ ]:
s2open.close()

### Visualizing Seurasaari

And we can open the cropped rasterfile again with rasterio and visualize the nir band. 

In [ ]:
seurasaariS2 = rasterio.open(seurasaariS2_file)

print(seurasaariS2.count)

# Visualize with colormap defined
show((seurasaariS2, 4), cmap="terrain");

Use the toolbox to prepare the image (as we did for Helsinki) and plot the fals color image:

In [ ]:
ss_falsecolorstack = stb.make_false_color_stack(seurasaariS2)

plt.imshow(ss_falsecolorstack)

Looks like there are a lot of trees on Seurasaari with high reflectances in the near infrared.



## Vegetatiom health indicator

The false color image does give us an impression about the vegetation on Seurasaari, but there is also other ways to look at "vegetation greenness/health".

One often used indicator for vegetation health is the [Normalized Difference Vegetation Index, NDVI](https://gisgeography.com/ndvi-normalized-difference-vegetation-index/) 

![](./img/ndvi.jpg)

Credit: Robert Simmon, NASA, https://www.nasa.gov/topics/earth/features/obscure_data.html

It is calculated as:

$$NDVI = \frac{nir-red}{nir+red}$$

### Reading the data

This formula is applied to every single pixel in the raster. So we will need to do some band math / map algebra. 
And since we can read our data into numpy arrays, let's utilize that and read the nir and red band into arrays. 
The toolbox provides the read_band function to read and prepare bands red (band 3) and near infrared (band 4):

In [ ]:
seurasaariS2 = rasterio.open(seurasaariS2_file)

ss_red = stb.read_band(seurasaariS2,3)
ss_nir = stb.read_band(seurasaariS2,4)

ss_red.dtype

Since our divisor can be 0, we choose to ignore that error from numpy:

In [ ]:
np.seterr(divide="ignore", invalid="ignore")

### Calculate NDVI

Then we can calculate the NDVI value for every single pixle in our Seurasaari raster using the formula from above:

In [ ]:
ndvi = (ss_nir - ss_red) / (ss_nir + ss_red)

### Visualize NDVI

Let's also visualize the NDVI using imshow (currently ndvi is only an array without geoinformation) with a colorbar:

In [ ]:
plt.imshow(ndvi, cmap="viridis")
plt.colorbar()

### Storing

Looks like the Seurasaari has been green and healthy. Let's store this ndvi array with added geoinformation as geotif. 
First we need again the metadata, which we can copy from the original file and overwrite the factors that changed.

In [ ]:
ndvi_meta = seurasaariS2.meta.copy()

ndvi_meta.update({"count" : 1,
                 "dtype" : np.dtype("f8")})

ndvi_file = "./data/S2_NDVI_Seurasaari.tif"

with rasterio.open(ndvi_file, "w", **ndvi_meta) as dest:
    dest.write_band(1, ndvi.astype(np.dtype("f8")))


**Check your understanding:**

* In above plot we do not have Coordinates at the plot, why? How could we get them?

In [ ]:
# %load cyu2_solution.txt

## Collecting more information

In general, it looks like the vegetation on Seurasaari is doing well. But what are those darker areas?

It is possible that not everything is trees on Seurasaari.

Let's check what else there is...

There is many possibilities to do that, we are going to look at the land cover classification [Corine](https://land.copernicus.eu/pan-european/corine-land-cover) , wich SYKE kindly provides for Finland: https://ckan.ymparisto.fi/dataset/%7B0B4B2FAC-ADF1-43A1-A829-70F02BF0C0E5%7D with the categories provided as excel sheet: https://geoportal.ymparisto.fi/meta/julkinen/dokumentit/CorineMaanpeite2018Luokat.xls

For this lesson we will use a cropped versio of the file provided by SYKE, covering Seurasaari.

### Visualizing CORINE

First, lets visualize Corine data of Seurasaari:

In [ ]:
corine_file = "./data/Clc2018_Seurasaari.tif"


with rasterio.open(corine_file) as corine:
    show(corine,1, cmap = "tab20b")

Up to now we have been working with continuous rasterdata representing reflectance between 0 and 1.
Corine is an example of a categorical raster, with each value representing a land cover class.
From the plot above we can see, that we have multiple different land cover classes on Seurasaari.

### Exploring land cover classes

Let's see what land cover classes we have on Seurasaari using `zonal_stats` function from [rasterstats](https://pythonhosted.org/rasterstats/).
Note that in this case we could also use numpy to collect values present in the corine array. `Zonal_stats` takes filenames as well as certain opened file formats.

In [ ]:
from rasterstats import zonal_stats

zstats = zonal_stats(seurasaari_df, corine_file, categorical=True)
zstats

For categorical rasters, `zonal_stats` output is a dictionary with category value (= land cover class) and number of pixels belonging to that class.
Now it would be great to know what these category values mean to find out what is forest. 
Luckily class descriptions are provided as excel file, of which we can get the interesting (english and detailed) description into a dictionary:

We can add this directly to zonal stats to get the output human reader friendly with classnames instead of class values (https://pythonhosted.org/rasterstats/manual.html#working-with-categorical-rasters). We can use the toolbox to get a dictionary of land cover class value and their name as category map:

In [ ]:
cat_dict = stb.get_corine_dict("https://geoportal.ymparisto.fi/meta/julkinen/dokumentit/CorineMaanpeite2018Luokat.xls")
zstats = zonal_stats(seurasaari_df,corine_file, categorical=True, category_map=cat_dict, stats=["count"])

We can get the pecentage of pixels for each class, by utilizing the 'count' statistics, which counts all valid pixels within the polygon. Use function from toolbox to calculate percentages for each class:

In [ ]:
stb.get_zonal_stats_percentage(zstats)

Now we know that there are commercial and industrial units on Seurasaari. Note that the Corine file was clipped approximately at the coast, which explains that there is 0.x% of sea and ocean and bare rock, even though if you have ever been to Seurasaari we know it is there.

### Extracting trees

But we are only interested in trees (forest in Corine case), so lets filter out only forest classes: 'Broad-leaved forest on mineral soil', 'Coniferous forest on mineral soil', 'Coniferous forest on rocky soil', 'Mixed forest on mineral soil', 'Mixed forest on rocky soil'. Let's get the codes of these classes.

In [ ]:
forestcodes = stb.get_forest_codes()

Let's create a mask which shows us where there is forest, and no forest on Seurasaari. 
First we have to read the Corinefile into a numpy array, it has only one band:

In [ ]:
with rasterio.open(corine_file) as corine:
    corine_array = corine.read(1)
    
#corine_array

Now we mask the corinearray everywhere where the values are equal to any of the forest codes:

In [ ]:
forestmask = stb.create_forest_mask(corine_array, forestcodes)

show(forestmask,1)

### Storing the forests as polygons

Now we store the resulting mask as a shapefile using [fiona](https://fiona.readthedocs.io/en/latest/manual.html) (a Python package for reading/writing geographic data files; geopandas would be another option) and [resterio.features](https://rasterio.readthedocs.io/en/latest/api/rasterio.features.html). We use features to get the values and connected regions in the mask and store them as polygons with the corresponding raster value as field. The polygons are then stored as shapefile with metadata from the corine raster.

In [ ]:
import fiona
from rasterio import features

results = ({"properties": {"raster_val": v}, "geometry": s} for i, (s, v) in enumerate(features.shapes(forestmask, transform = corine.transform)))

forestmask_file = "./data/forestmask_corine_seurasaari2.shp"

with fiona.open(forestmask_file, "w", driver="Shapefile", crs=corine.crs, schema={"properties": [("raster_val", "int")], "geometry": "Polygon"}) as dst:
    dst.writerecords(results)

We are only interested in the health of forests, so we can use the created shapefile to mask out non-forest areas in the NDVI raster. This leads us to having one value to indicate "how healthy the trees on Seurasaari were in end of September 2021".

First we open the Seurasaari NDVI file and use rasterio mask to mask out areas that are not forest. We only want to regard polygons that have the value 1 (forest) that we can get from the raster_val field

In [ ]:
with rasterio.open(ndvi_file) as ndvi:
    shapefilename = stb.get_reprojected_shapefilename(ndvi.crs, "./data/forestmask_corine_seurasaari2.shp")
    
    with fiona.open(shapefilename, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile if feature["properties"]["raster_val"] == 1]
    
    ndvi_forest, ndvi_forest_transform = rasterio.mask.mask(ndvi, shapes, crop=True, nodata=np.nan)

print("Mean NDVI value of Seurasaari forests in September 2021:")
print(np.around(np.nanmean(ndvi_forest),2))

Seems like the trees on Seurasaari are green and probably rather healthy :)

## Keypoints

* You can do almost anything to geospatial rasters with rasterio
* Single bands can be read into numpy arrays and handled as such for e.g band maths
* Be aware when you are working with geocoded data and when you are working with arrays
* You can use rasterio mask to crop a rasterfile with a shapefile
* Rasterstats.zonal_statistics is helpful when you want to summarize pixel values from categorical and continuous rasters within polygons

## References

Inspiration for this lesson was drawn from below sources which are also good resources for further reading:

* Original Raster lesson of this course: https://autogis-site.readthedocs.io/en/latest/lessons/Raster/overview.html (check especially Mosaicing)
* Raster data in Python intros: 
    * https://www.earthdatascience.org/courses/use-data-open-source-python/intro-raster-data-python/
    * https://carpentries-incubator.github.io/geospatial-python/05-raster-structure/
    * https://geohackweek.github.io/raster/04-workingwithrasters/
    * https://kodu.ut.ee/~kmoch/geopython2019/L4/raster.html (check out especially the part about reprojecting a raster)
    * https://geoscripting-wur.github.io/PythonRaster/
    * https://pysal.org/scipy2019-intermediate-gds/deterministic/gds2-rasters.html
    * https://geobgu.xyz/py/rasterio.html
    * https://snowex-hackweek.github.io/website/tutorials/geospatial/raster.html
    * https://carpentries-incubator.github.io/geospatial-python/01-intro-raster-data/index.html
* Sample raster values at specific coordinates: https://hackernoon.com/sampling-raster-values-at-specific-coordinates-with-python
* Virtual Rasters: https://docs.csc.fi/support/tutorials/gis/virtual-rasters/
* Great collection of Geospatial Python resources: https://github.com/giswqs/python-geospatial